In [4]:
import pandas as pd

# Load EC40 Dataset

In [6]:
ec40 = pd.read_pickle('../dataset/ec40/ec40.pkl')
ec40[:50]

,accession,sequence,ec,traintest,negative_for,mainclass_set,sprot_version,len
0,Q7VRM4,MQAKILRIATRKSPLAICQACYVCNKLKHYHPHIQTELIPIITTGD...,[2.5.1.61],0,NaN,Transferases,2017_03,1
1,A4XK06,MKKLRIGARDSKLSRIQVDIVARKIKQTLGIECEFVPIKTKGDIDK...,[2.5.1.61],0,NaN,Transferases,2017_03,1
2,Q8KCJ4,MKKELIIGTRSSPLALWQAEFTKAELSRHFPELNITLKLVKTTGDV...,[2.5.1.61],0,NaN,Transferases,2017_03,1
3,Q9VR91,MFNRQASGGAGSSGQGAGSSQTASAAPVSAGVGVGGGGGASGAAAG...,[2.3.2.26],0,NaN,Transferases,2017_03,1
4,O95714,MPSESFCLAAQARLDSKWLKTDIQLAFTRDGLCGLWNEMVKDGEIV...,[2.3.2.26],0,NaN,Transferases,2017_03,1
5,F2Z461,MYFSWAAGSRKPRRLKAGTSGIELLQAASGEHHSLLLFSNHRVYSC...,[2.3.2.-],0,NaN,Transferases,2017_03,1
6,O66129,MIALSYKAFLNPYIIEVEKRLYECIQSDSETINKAAHHILSSGGKR...,[2.5.1.83],0,NaN,Transferases,2017_03,1
7,Q68CP4,MTGARASAAEQRRAGRSGQARAAERAAGMSGAGRALAALLLAASVL...,[2.3.1.78],0,NaN,Transferases,2017_03,1
8,P40235,MALDLRIGNKYRIGRKIGSGSFGDIYLGTNVVSGEEVAIKLESTRA...,[2.7.11.1],0,NaN,Transferases,2017_03,1
9,P23874,MPKLVTWMNNQRVGELTKLANGAHTFKYAPEWLASRYARPLSLSLP...,[2.7.11.1],0,NaN,Transferases,2017_03,1


# Prepare test sequence

In [9]:
# Filter for test sequences
# (Adjust the filtering criteria if your CSV uses a different convention;
#  here we assume '0' indicates test sequences.)
test_df = ec40[ec40["traintest"] == 0][:50]

print(f"Found {len(test_df)} test sequences.")

# Write the test sequences to a FASTA file.
# We will use the 'accession' column as the FASTA header and 'sequence' as the sequence.
with open("../dataset/test_sequences/test_sequences.fasta", "w") as fout:
    for index, row in test_df.iterrows():
        accession = row["accession"]
        sequence = row["sequence"]
        fout.write(f">{accession}\n{sequence}\n")

Found 50 test sequences.


# Dimond Query

Download DIMOND
+ if not downloaded, uncomment below
+ for macos: use homebrew
+ for windows: TBC

In [22]:
# !brew install diamond

Prepare DIMOND Database Folder
+ if not downloaded, uncomment below

In [23]:
# !mkdir ../dataset/dimond_db/
# %cd ../dataset/dimond_db/

Download DIMOND Database
+ if not downloaded, uncomment below

In [24]:
# !wget ftp://ftp.uniprot.org/pub/databases/uniprot/current_release/uniref/uniref90/uniref90.fasta.gz
# %cd ../../experiments

Generate Database from Fasta

In [25]:
# !diamondmakedb --in ../dataset/dimond_db/uniref90.fasta -d ../dataset/dimond_db/uniref90.dmnd

Run DIMOND search
+ May take 10mins for 50 query

In [11]:
!diamond blastp --db ../dataset/dimond_db/uniref90.dmnd --query ../dataset/test_sequences/test_sequences.fasta --out ../dataset/test_sequences/test_sequences_results.m8 --threads 16

diamond v2.1.11.165 (C) Max Planck Society for the Advancement of Science, Benjamin Buchfink, University of Tuebingen
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 16
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Temporary directory: ../dataset/test_sequences
#Target sequences to report alignments for: 25
Opening the database...  [0.07s]
Database: ../dataset/dimond_db/uniref90.dmnd (type: Diamond database, sequences: 199553294, letters: 68521788531)
Block size = 2000000000
Opening the input file...  [0s]
Opening the output file...  [0s]
Loading query sequences...  [0s]
Masking queries...  [0.006s]
Building query seed set...  [0.003s]
Algorithm: Query-indexed
Building query histograms...  [0.001s]
Seeking in database...  [0s]
Loading reference sequences...  [5.767s]
Initializing dictionary...  [0.056s]
Initializing temporary storage...